In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from common.models import simple_conv
import numpy as np



c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
bs = 4
tl = 1024
in_feats = 460
base_width = 128
down_layers = 3
dev = torch.device("cuda")

encoder = simple_conv.Encoder(in_feats, down_layers, base_width, [False, True, True])

# model = simple_conv.SimpleSegConv(encoder, 64)
model = simple_conv.SimpleConv(encoder, 64, 1024).to(dev)


In [4]:
ins = torch.randn(bs, tl, in_feats, device=dev)
ins.size()

torch.Size([4, 1024, 460])

In [7]:
_res = []
for _ in range(100):
    _res.extend(model(ins).cpu().detach().numpy())
res = np.concatenate(_res)

In [8]:
res.shape

(409600, 64)

In [9]:
from sklearn.neighbors import KernelDensity

n, d = res.shape
h = (n * (d + 2) / 4.)**(-1. / (d + 4))

kde = KernelDensity(kernel="gaussian", bandwidth=h)

In [10]:
kde.fit(res)

KernelDensity(bandwidth=0.7935259235952904)

In [12]:
from common.kde import KernelDensityEstimator, GaussianKernel

kde2 = KernelDensityEstimator(torch.as_tensor(res), GaussianKernel(h), dev)

In [13]:
kde.score_samples(res[:10])

array([-51.86289462, -51.20264501, -50.40261673, -50.46027711,
       -51.00540989, -50.88424085, -50.40550028, -50.18489944,
       -49.66279666, -49.59571227])

In [17]:
kde2.score_samples(torch.as_tensor(res))

tensor(0)